# Simulation Models
This notebook explains how a simulation may be set up combining several types of classes into a "Model". We first sketch the high-level concepts and structure, before demonstrating basic usage, i.e., adjusting existing models to specific applications. Then, we turn to more advanced usage such as development of new models and more fundamental adjustment of existing ones.
## Structure and concepts
The essential parts needed to set up a simulation are the following class types:

* Equations
* Constitutive laws
* Variables
* Boundary conditions
* Solution strategy

The first four define the problem, whereas the last one specifies how to solve it. Additionally, values for constant material parameters are provided as `MaterialConstant` classes and visualization is sorted using DataSavingMixin.